In [1]:
import preprocess.data
import models
import importlib
import util
importlib.reload(util)
importlib.reload(preprocess.data)
importlib.reload(models)
from models import BoW,CNN_Text
from optimizer import AdamW
from preprocess.data import load_mldoc_dataset
from preprocess.data import generate_features
from preprocess.moses import MosesTokenizer
from preprocess.japanese_tokenizer import JapaneseTokenizer
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import numpy as np
from gensim.models import KeyedVectors
from collections import defaultdict
import random
import copy

In [2]:
device = torch.device('cuda')
dataset_path = "/home/smg/nishikawa/corpus"
data = load_mldoc_dataset(dataset_path,"en",dev_size=0.05,seed=1)
tokenizer = MosesTokenizer(lang="en")
ret = generate_features(data, tokenizer, 3, 500)
word_vocab = ret['word_vocab']

100%|██████████| 6000/6000 [00:55<00:00, 108.04it/s]


In [5]:
vec_path = "/home/smg/nishikawa/adversarial-examples-experiments/data/en.50k.vec"

In [6]:
embedding = KeyedVectors.load_word2vec_format(vec_path, binary=False)

In [31]:
dim_size = embedding.syn0.shape[1]
cnt = 0
word_embedding = np.random.uniform(low=-0.05, high=0.05, size=(len(word_vocab), dim_size))
word_embedding[0] = np.zeros(dim_size)
for word,index in word_vocab.items():
    try:
        word_embedding[index] = embedding.wv[word]
        cnt += 1
    except KeyError:
        continue
print(cnt)

/home/smg/nishikawa/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.
/home/smg/nishikawa/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


16308


In [8]:
def evaluate(model, data_loader, device, fold):
    model.eval()

    predictions = []
    labels = []
    with torch.no_grad():
        for batch in data_loader:
            args = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            logits = model(**args)
            predictions += torch.argmax(logits, 1).to('cpu').tolist()
            labels += batch['label'].to('cpu').tolist()

    test_acc = accuracy_score(labels, predictions)
    print(f'accuracy ({fold}): {test_acc:.4f}')

    test_f1 = f1_score(labels, predictions, average='macro')
    print(f'f-measure ({fold}): {test_f1:.4f}')

    return test_acc, test_f1

In [9]:
batch_size = 16
dropout_prob = 0.1
weight_decay = 0.01
warmup_epochs = 2
learning_rate = 1e-4
train_data_loader = DataLoader(ret['train'], shuffle=False, batch_size=batch_size)
dev_data_loader = DataLoader(ret['dev'], shuffle=False, batch_size=batch_size)
test_data_loader = DataLoader(ret['test'], shuffle=False, batch_size=batch_size)

In [10]:
model = CNN_Text(word_embedding, len(data.label_names), dropout_prob)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
model.to(device)

CNN_Text(
  (word_embedding): Embedding(25625, 300)
  (convs1): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.1)
  (fc1): Linear(in_features=300, out_features=4, bias=True)
)

In [11]:
epoch = 0
patience = 5
best_val_acc = 0.0
best_weights = None
num_epochs_without_improvement = 0
while True:
    with tqdm(train_data_loader) as pbar:
        model.train()
        for batch in pbar:
            args = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            logits = model(**args)
            loss = F.cross_entropy(logits, torch.tensor(batch['label']).to(device))
            loss.backward(retain_graph=True)
            optimizer.step()
            model.zero_grad()
            pbar.set_description(f'epoch: {epoch} loss: {loss.item():.8f}')

    epoch += 1
    val_acc = evaluate(model, dev_data_loader, device, 'dev')[0]
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_weights = {k: v.to('cpu').clone() for k, v in model.state_dict().items()}
        num_epochs_without_improvement = 0
    else:
        num_epochs_without_improvement += 1

    if num_epochs_without_improvement >= patience:
        model.load_state_dict(best_weights)
        break

  0%|          | 0/179 [00:00<?, ?it/s]/home/smg/nishikawa/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
epoch: 1 loss: 0.84180301:   3%|▎         | 6/179 [00:00<00:03, 55.35it/s]

accuracy (dev): 0.7133
f-measure (dev): 0.6973


epoch: 2 loss: 0.68079937:   3%|▎         | 6/179 [00:00<00:03, 56.31it/s]

accuracy (dev): 0.7667
f-measure (dev): 0.7593


epoch: 3 loss: 0.66905886:   3%|▎         | 6/179 [00:00<00:03, 56.25it/s]

...

accuracy (dev): 0.8600
f-measure (dev): 0.8554


epoch: 15 loss: 0.34444413:   3%|▎         | 6/179 [00:00<00:03, 56.29it/s]

accuracy (dev): 0.8533
f-measure (dev): 0.8477


epoch: 15 loss: 0.37427598: 100%|██████████| 179/179 [00:03<00:00, 55.87it/s]


accuracy (dev): 0.8533
f-measure (dev): 0.8477


In [12]:
# make distillation train_data

In [13]:
model.zero_grad()
teacher_logits = []
answer = []
cnt = 0
with torch.no_grad():
    with tqdm(train_data_loader) as pbar:
        model.eval()
        for batch in pbar:
            args = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            logits = model(**args)
            teacher_logits.append(logits)
            answer.append(torch.tensor(batch['label']).to(device))
        
#         loss = F.cross_entropy(logits, torch.tensor(batch['label']).to(device))


  0%|          | 0/179 [00:00<?, ?it/s]/home/smg/nishikawa/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
100%|██████████| 179/179 [00:01<00:00, 141.50it/s]


In [187]:
model_student = CNN_Text(word_embedding, len(data.label_names), dropout_prob)
optimizer = optim.Adam(model_student.parameters(), lr=learning_rate, weight_decay=weight_decay)
model_student.to(device)

CNN_Text(
  (word_embedding): Embedding(25625, 300)
  (convs1): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(5, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.1)
  (fc1): Linear(in_features=300, out_features=4, bias=True)
)

In [15]:
def softmax(x): return x.exp() / (x.exp().sum(-1)).unsqueeze(-1)
def nl(input, target): return -input[range(target.shape[0]), target].log().mean()

In [17]:
def cross_entropy_for_distillation(student_logit, teacher_logit, temp=1.0): 
    student = F.softmax(student_logit / temp, dim = -1)
    teacher = F.softmax(teacher_logit / temp, dim = -1)
#     return - (student.log() * teacher).mean()
    return - (student.log() * teacher).mean()

In [189]:
epoch = 0
patience = 3
best_val_acc = 0.0
best_weights = None
num_epochs_without_improvement = 0
alpha = 1
temp = 10
while True:
    with tqdm(train_data_loader) as pbar:
        model_student.train()
        for i, batch in enumerate(pbar):
#             print(torch.tensor(batch['label']).to(device))
#             break
            args = {k: v.to(device) for k, v in batch.items() if k != 'label'}
            logits = model_student(**args)
            
#             pred = softmax(logits)
#             loss=nl(pred, answer[i])

            loss_soft = cross_entropy_for_distillation(logits, teacher_logits[i], temp)
            loss_hard = F.cross_entropy(logits, torch.tensor(batch['label']).to(device))
#             loss = loss_soft * alpha * (temp**2) + loss_hard * (1-alpha)
            loss = loss_soft * alpha + loss_hard * (1-alpha)
#             loss = loss_soft
            loss.backward(retain_graph=True)
            optimizer.step()
#             print(model_student.fc1.weight)
            model_student.zero_grad()
            pbar.set_description(f'epoch: {epoch} loss: {loss.item():.8f}')

    epoch += 1
    val_acc = evaluate(model_student, dev_data_loader, device, 'dev')[0]
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_weights = {k: v.to('cpu').clone() for k, v in model_student.state_dict().items()}
        num_epochs_without_improvement = 0
    else:
        num_epochs_without_improvement += 1

    if num_epochs_without_improvement >= patience:
        model_student.load_state_dict(best_weights)
        break
#     break

epoch: 1 loss: 0.34435105:   3%|▎         | 6/179 [00:00<00:03, 57.54it/s]

accuracy (dev): 0.8600
f-measure (dev): 0.8563


epoch: 2 loss: 0.34531435:   3%|▎         | 6/179 [00:00<00:03, 55.24it/s]

accuracy (dev): 0.8600
f-measure (dev): 0.8571


epoch: 3 loss: 0.34548065:   3%|▎         | 6/179 [00:00<00:03, 55.03it/s]

accuracy (dev): 0.8600
f-measure (dev): 0.8571


epoch: 3 loss: 0.34467620: 100%|██████████| 179/179 [00:03<00:00, 54.66it/s]


accuracy (dev): 0.8333
f-measure (dev): 0.8289


In [190]:
model.word_embedding = nn.Embedding(word_embedding.shape[0], word_embedding.shape[1], padding_idx=0)
model.word_embedding.weight = nn.Parameter(torch.FloatTensor(word_embedding))
model.to(device)
model_student.word_embedding = nn.Embedding(word_embedding.shape[0], word_embedding.shape[1], padding_idx=0)
model_student.word_embedding.weight = nn.Parameter(torch.FloatTensor(word_embedding))
model_student.to(device)
print(evaluate(model, test_data_loader, device, 'test')[0])
print(evaluate(model_student, test_data_loader, device, 'test')[0])

accuracy (test): 0.8853
f-measure (test): 0.8842
0.8853333333333333
accuracy (test): 0.8783
f-measure (test): 0.8772
0.8783333333333333


In [20]:
cont_words = torch.zeros(len(word_vocab)).to(device)

In [21]:
epoch = 0
# model.word_embedding.weight.requires_grad=True
with tqdm(train_data_loader) as pbar:
    model.train()
    for batch in pbar:
        args = {k: v.to(device) for k, v in batch.items() if k != 'label'}
        logits = model(**args)
        loss = F.cross_entropy(logits, torch.tensor(batch['label']).to(device))
        loss.backward()
        grads = model.word_embedding.weight.grad
        cont_words += torch.norm(grads,dim=-1)
        model.zero_grad()

  0%|          | 0/179 [00:00<?, ?it/s]/home/smg/nishikawa/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 179/179 [00:02<00:00, 81.21it/s]


In [29]:
epoch = 0
# model.word_embedding.weight.requires_grad=True
with tqdm(train_data_loader) as pbar:
    model_student.train()
    for batch in pbar:
        args = {k: v.to(device) for k, v in batch.items() if k != 'label'}
        logits = model_student(**args)
        loss = F.cross_entropy(logits, torch.tensor(batch['label']).to(device))
        loss.backward()
        grads = model_student.word_embedding.weight.grad
        cont_words += torch.norm(grads,dim=-1)
        model_student.zero_grad()

  0%|          | 0/2375 [00:00<?, ?it/s]/home/smg/nishikawa/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
100%|██████████| 2375/2375 [00:30<00:00, 76.74it/s]


In [22]:
cont_words[:100]

tensor([0.0000e+00, 5.4424e-03, 1.0888e+00, 6.1385e-02, 2.7390e-01, 9.6517e-01,
        3.5304e-01, 2.0702e-01, 4.2818e-03, 1.2990e+00, 9.6241e-02, 2.8942e-01,
        1.7207e-02, 1.5194e+00, 5.2167e-02, 2.5741e+00, 3.8088e-01, 1.3754e-01,
        2.3049e+00, 1.1720e+00, 1.7243e-01, 1.8760e-01, 1.8355e-01, 3.0804e-01,
        1.5545e-01, 1.3924e+00, 4.9646e-02, 7.0652e-02, 2.3712e-01, 5.2886e-02,
        8.3334e-01, 6.6389e-04, 6.3035e-01, 2.0506e-01, 8.2625e-03, 1.6211e-01,
        4.2441e-02, 1.9975e+00, 2.6601e-01, 1.2476e-02, 6.9769e-02, 2.1281e-02,
        8.1884e-02, 5.8429e-02, 2.5545e-03, 9.7494e-03, 3.1470e-04, 1.0824e-01,
        1.1977e-03, 6.2149e-02, 6.0036e-02, 3.0235e-01, 0.0000e+00, 1.0331e-03,
        1.5695e-02, 1.8593e-01, 1.7621e-03, 8.3865e-02, 6.1081e-02, 6.3769e-06,
        5.4858e-01, 4.4668e-03, 5.9412e-06, 1.2524e-02, 8.1610e-01, 4.5557e-01,
        3.4215e-03, 1.2397e-01, 9.2432e-01, 6.2447e-01, 2.1389e-02, 2.1474e-02,
        1.2341e+00, 2.5277e-03, 4.4823e-

In [25]:
imp_inds[:100]

tensor([ 695,   15,   18,   37,   88, 1958,  162,   13,  684, 1034,   25,  257,
        1097,    9,  339,   72,  195,   19,  166,  276, 1252,    2,  432,   94,
        3505,  557,    5,  111,   68,  369,  164,  771,   83,   30,   64,  804,
         724,  320, 2343, 3250,  481,   97,  602,  299,  675,  191, 1096,  681,
         772,  488,   32,  372, 2426,   69, 1480, 1773,  286, 1304, 1311,  456,
        2810,   60,  189,  228,  887, 1367, 5644,  762,  234,  403, 1269, 2463,
          65,  498,  982,   74, 1518, 2675, 4015, 2906,  718,  694,  449, 1386,
         319,   89,  412, 1292,  198,  136,   16, 1322,  265, 2216,    6,  470,
         328,   93,  251,  117], device='cuda:0')

In [26]:
imp_inds_to_rank = dict()
for i,imp_ind in enumerate(imp_inds):
    imp_inds_to_rank[imp_ind.item()] = i

In [114]:
word_vocab_r = {v:k for k,v in word_vocab.items()}

In [119]:
word_vocab_r[88]

'said'

In [24]:
imp_inds = torch.argsort(cont_words, descending=True)

In [31]:
word_vocab_r = {v:k for k,v in word_vocab.items()}

In [28]:
imp_inds = set([id.item() for id in imp_inds[:1000]])

In [29]:
dim_size = embedding.syn0.shape[1]
cnt = 0
word_embedding = np.random.uniform(low=-0.05, high=0.05, size=(len(word_vocab), dim_size))
word_embedding[0] = np.zeros(dim_size)
for word,index in word_vocab.items():
    if index in imp_inds:
        continue
    try:
        word_embedding[index] = embedding.wv[word]
        cnt += 1
    except KeyError:
        continue
print(cnt)

/home/smg/nishikawa/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.
/home/smg/nishikawa/.pyenv/versions/miniconda3-4.7.12/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':


15326


In [169]:
model.word_embedding = nn.Embedding(word_embedding.shape[0], word_embedding.shape[1], padding_idx=0)
model.word_embedding.weight = nn.Parameter(torch.FloatTensor(word_embedding))
model.to(device)
model_student.word_embedding = nn.Embedding(word_embedding.shape[0], word_embedding.shape[1], padding_idx=0)
model_student.word_embedding.weight = nn.Parameter(torch.FloatTensor(word_embedding))
model_student.to(device)
evaluate(model, test_data_loader, device, 'test')[0]
evaluate(model_student, test_data_loader, device, 'test')[0]

accuracy (test): 0.8853
f-measure (test): 0.8842
accuracy (test): 0.8630
f-measure (test): 0.8612


0.863

In [34]:
with tqdm(test_data_loader) as pbar:
    for batch in pbar:
        print(batch['word_ids'])
        break

  0%|          | 0/188 [00:00<?, ?it/s]

tensor([[22726,    18,  4086,  ...,     0,     0,     0],
        [  574,  4543,     2,  ...,     0,     0,     0],
        [   15,    28,  4015,  ...,     0,     0,     0],
        ...,
        [ 4947,  7791,  8733,  ...,     0,     0,     0],
        [ 1729,    65,  4083,  ...,     0,     0,     0],
        [10594, 10595,  5807,  ...,     0,     0,     0]])


In [35]:
batch['word_ids'][0]

tensor([22726,    18,  4086,   804,   282,    13,  2476,  2477,  2449,  2478,
         7655,  4809,    18,     6,   501,    25,   725,   307,    18,    19,
         2476,   853,    88,    68,    69,    15,  2241,   166,   504,   649,
         1394,   198,  2476,   189,  2759,  9700,   198,  1462,  4086,   804,
         2476,  4809,    18,  3831,   384,   113,   184,    18,    69,    15,
          853,   117,    87,    65,  5455,   166,    69,  1324,   237,  5043,
           69,    18,    60,  1387,    15,   853,   614,    89,   111,     2,
         2556, 22726,   109,  1730,   166,  3517,    25,  3429,   725,   307,
           18,   166,  6821,    15,  4809,  1288,     9,  3304, 13133,   166,
           19,  2152,   905,  5506, 22726,     6,     7,  7888,   166,  2208,
         3779,   730,    15,  3244,    13,    15, 13019,  9762,    18,  1789,
         4086,   804,   282,    18,     6,  1413,     9, 22726,     2,  2975,
           18,    15,   853,  1387,  1673,   807,    88, 22726, 

In [77]:
ret['test'][0]

{'word_ids': array([22726,    18,  4086,   804,   282,    13,  2476,  2477,  2449,
         2478,  7655,  4809,    18,     6,   501,    25,   725,   307,
           18,    19,  2476,   853,    88,    68,    69,    15,  2241,
          166,   504,   649,  1394,   198,  2476,   189,  2759,  9700,
          198,  1462,  4086,   804,  2476,  4809,    18,  3831,   384,
          113,   184,    18,    69,    15,   853,   117,    87,    65,
         5455,   166,    69,  1324,   237,  5043,    69,    18,    60,
         1387,    15,   853,   614,    89,   111,     2,  2556, 22726,
          109,  1730,   166,  3517,    25,  3429,   725,   307,    18,
          166,  6821,    15,  4809,  1288,     9,  3304, 13133,   166,
           19,  2152,   905,  5506, 22726,     6,     7,  7888,   166,
         2208,  3779,   730,    15,  3244,    13,    15, 13019,  9762,
           18,  1789,  4086,   804,   282,    18,     6,  1413,     9,
        22726,     2,  2975,    18,    15,   853,  1387,  1673,  

In [175]:
adv_test = []
for test_data in tqdm(ret['test']):
    rank_list = [imp_inds_to_rank[input_id] for input_id in test_data['word_ids'][test_data['word_ids']!=0]]
    index_list = np.argsort(rank_list)
    adv = copy.deepcopy(test_data['word_ids'])
#     print(int(len(index_list) * 0.1))
    adv[index_list[:int(len(index_list) * 0.7)]] = 0
    adv_data = dict()
    adv_data['word_ids'] = adv
    adv_data['label'] = test_data['label']
    adv_test.append(adv_data)
#     break

        

100%|██████████| 3000/3000 [00:00<00:00, 4031.21it/s]


In [153]:
id_list

[149,
 25,
 123,
 14,
 120,
 86,
 61,
 6,
 5,
 84,
 46,
 125,
 49,
 23,
 108,
 71,
 88,
 64,
 66,
 135,
 81]

In [159]:
for inputid in adv[index_list[:int(len(index_list) * 0.1)]]:
    print(word_vocab_r[inputid])

<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>
<PAD>


In [140]:
a = "Stan Wraight, vice president sales of KLM Royal Dutch Airlines NV Cargo, will leave on May 1, a KLM statement said. "The decision to end his service with KLM was taken jointly with executive vice president KLM Cargo, Jacques Ancher some time ago," the statement said. He is leaving to "pursue other interests", it added. The statement added that at Ancher's request Wraight had agreed to stay on until May 1, to assist the cargo division in completing transformation to a full unit structure. Wraight will be returning to North America following the completion of the restructuring. Paul Elich, deputy vice president sales, will act in Wraight's place, the statement added. Industry sources said Wraight had been tipped to succeed Ancher. He joined KLM Cargo Handling department at Montreal Dorval Airport in 1965. Wraight was appointed KLM Cargo sales manager UK & Ireland in 1984 and in 1992 he became vice president and cargo area manager North and East Asia, Australia. He remained in this position until he took on his last role with the company as vice president cargo sales in 1995. -Reuters Air Cargo Newsroom Tel+44 171 542-7706 Fax+5017"

SyntaxError: invalid syntax (<ipython-input-140-5c1038c6b640>, line 1)

In [122]:
a

array([2, 0, 1])

In [127]:
adv_test[1]

{'word_ids': array([  574,  4543,     2,   791,  1101,   872,  1886,  6086,  8997,
          166,    25,   854,   620,   166, 14195,    19, 22706,   872,
          166,  1014,     0,  1737,  1790,  1084,     0,  4548,  4549,
          166,  1729,     9,   755,  1706,  4787,     0,  4548,   229,
          166,  1729,   111,  4550,    25,   739,   205,    18,  1749,
           19,  3335,    72,    19,   302,    13,  5264,  4178,  3242,
           51,    88,     0,  3294,   795,  1778,   574,  4543,   418,
            0,  4544,     6,     7,  4995,   234,    19, 12476,   196,
            0, 14823,  8988,     9,   574,  4543,    51,  3271,     0,
         1020,  1865,    11,    72,     0,   173,     0,  2066,    98,
            7,  3330,   234,    19,  5231,   494,     0,    33,    51,
            0,  4954,   128,     9,  4544,  9234,  6086,    23,    64,
         6406,    89,    60,    98,  7626,   546,   574,  4543,    72,
         1729,   464,     0,   296,   872,   189,   620,  1557,  

In [176]:
adv_data_loader = DataLoader(adv_test, shuffle=False, batch_size=batch_size)

In [191]:
evaluate(model, adv_data_loader, device, 'test')[0]
evaluate(model_student, adv_data_loader, device, 'test')[0]

accuracy (test): 0.5837
f-measure (test): 0.5563
accuracy (test): 0.5803
f-measure (test): 0.5484


0.5803333333333334

In [88]:
index_list = [imp_inds_to_rank[input_id] for input_id in test_data['word_ids'][test_data['word_ids']!=0]]

In [107]:
word_vocab

{'tanzania': 1,
 '&apos;s': 2,
 'largest': 3,
 'state-owned': 4,
 'bank': 5,
 'will': 6,
 'be': 7,
 'split': 8,
 'in': 9,
 'three': 10,
 'as': 11,
 'part': 12,
 'of': 13,
 'reforms': 14,
 'the': 15,
 'financial': 16,
 'sector': 17,
 ',': 18,
 'a': 19,
 'senior': 20,
 'central': 21,
 'official': 22,
 'told': 23,
 'reuters': 24,
 'on': 25,
 'tuesday.': 26,
 'reform': 27,
 'national': 28,
 'commerce': 29,
 '(': 30,
 'nbc': 31,
 ')': 32,
 'which': 33,
 'controls': 34,
 'over': 35,
 '70': 36,
 'percent': 37,
 'banking': 38,
 'carried': 39,
 'out': 40,
 'within': 41,
 'next': 42,
 'six': 43,
 'months.': 44,
 'established': 45,
 '1967': 46,
 'under': 47,
 'sweeping': 48,
 'socialist': 49,
 'policies': 50,
 'has': 51,
 '172': 52,
 'branches': 53,
 'across': 54,
 'this': 55,
 'impoverished': 56,
 'east': 57,
 'african': 58,
 'nation.': 59,
 'it': 60,
 'operates': 61,
 'semi-autonomous': 62,
 'although': 63,
 'government': 64,
 'is': 65,
 'sole': 66,
 'shareholder': 67,
 '.': 68,
 '&quot;': 69,


In [59]:
index_list = [imp_inds_to_rank[input_id] for input_id in ret['test'][0]['word_ids']]

In [61]:
index_list.sort()

In [67]:
index_list[:int(len(index_list) * 0.1)]

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 4,
 4,
 7,
 7,
 10,
 10,
 10,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 15,
 15,
 15,
 17,
 17,
 18,
 18,
 18,
 18,
 18,
 18,
 18,
 21,
 21,
 27,
 27]

In [70]:
adv = copy.deepcopy(ret['test'][0]['word_ids'])

In [72]:
adv[index_list[:int(len(index_list) * 0.1)]] = 0

In [73]:
adv

array([    0,     0,     0,   804,     0,    13,  2476,     0,  2449,
        2478,     0,  4809,    18,     0,   501,     0,   725,     0,
           0,    19,  2476,     0,    88,    68,    69,    15,  2241,
           0,   504,   649,  1394,   198,  2476,   189,  2759,  9700,
         198,  1462,  4086,   804,  2476,  4809,    18,  3831,   384,
         113,   184,    18,    69,    15,   853,   117,    87,    65,
        5455,   166,    69,  1324,   237,  5043,    69,    18,    60,
        1387,    15,   853,   614,    89,   111,     2,  2556, 22726,
         109,  1730,   166,  3517,    25,  3429,   725,   307,    18,
         166,  6821,    15,  4809,  1288,     9,  3304, 13133,   166,
          19,  2152,   905,  5506, 22726,     6,     7,  7888,   166,
        2208,  3779,   730,    15,  3244,    13,    15, 13019,  9762,
          18,  1789,  4086,   804,   282,    18,     6,  1413,     9,
       22726,     2,  2975,    18,    15,   853,  1387,  1673,   807,
          88, 22726,

In [48]:
a = np.ones((100))

In [54]:
a[[1,10]] = [4., 3.]

In [55]:
a

array([1., 4., 3., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [58]:
imp_inds_to_rank

{695: 0,
 15: 1,
 18: 2,
 37: 3,
 88: 4,
 1958: 5,
 162: 6,
 13: 7,
 684: 8,
 1034: 9,
 25: 10,
 257: 11,
 1097: 12,
 9: 13,
 339: 14,
 72: 15,
 195: 16,
 19: 17,
 166: 18,
 276: 19,
 1252: 20,
 2: 21,
 432: 22,
 94: 23,
 3505: 24,
 557: 25,
 5: 26,
 111: 27,
 68: 28,
 369: 29,
 164: 30,
 771: 31,
 83: 32,
 30: 33,
 64: 34,
 804: 35,
 724: 36,
 320: 37,
 2343: 38,
 3250: 39,
 481: 40,
 97: 41,
 602: 42,
 299: 43,
 675: 44,
 191: 45,
 1096: 46,
 681: 47,
 772: 48,
 488: 49,
 32: 50,
 372: 51,
 2426: 52,
 69: 53,
 1480: 54,
 1773: 55,
 286: 56,
 1304: 57,
 1311: 58,
 456: 59,
 2810: 60,
 60: 61,
 189: 62,
 228: 63,
 887: 64,
 1367: 65,
 5644: 66,
 762: 67,
 234: 68,
 403: 69,
 1269: 70,
 2463: 71,
 65: 72,
 498: 73,
 982: 74,
 74: 75,
 1518: 76,
 2675: 77,
 4015: 78,
 2906: 79,
 718: 80,
 694: 81,
 449: 82,
 1386: 83,
 319: 84,
 89: 85,
 412: 86,
 1292: 87,
 198: 88,
 136: 89,
 16: 90,
 1322: 91,
 265: 92,
 2216: 93,
 6: 94,
 470: 95,
 328: 96,
 93: 97,
 251: 98,
 117: 99,
 2592: 100,
 1